# Advanced Example: Dynamic Hedging with Incomplete Information
Fill me in

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Task 1: Load simulation data from the worked example
Before setting up the hedge on the `TSLA` put option, let's load the simulation state file we generated in the worked example. To load the state file, we will use [the `load(...)` method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl), and we will provide the path to the save file as an argument in the `path_to_save_file::String` variable:

In [5]:
path_to_save_file = joinpath(_PATH_TO_DATA, "CHEME-134-M4-WorkedExample-State.jld2");

Then, use [`load(...)` method](https://github.com/JuliaIO/JLD2.jl) to load the file and return a dictionary with the keys specified in the worked example; we save this dictionary in the `my_simulation_dict::Dict{String, Any}` variable:

In [7]:
my_simulation_dict = load(path_to_save_file)

Dict{String, Any} with 14 entries:
  "risk_free_rate"                   => 0.0431
  "T"                                => 0.165753
  "number_of_samples"                => 1000
  "h"                                => 366
  "DTE"                              => 60.5
  "samples_share_price"              => [261.49 261.49 … 261.49 261.49; 261.308…
  "sample_synthetic_index"           => 568
  "samples_IV"                       => [0.605621 0.605621 … 0.605621 0.605621;…
  "perfect_hedge_simulation_example" => Dict{Int64, NamedTuple}(5=>(shares = 43…
  "Δt"                               => 1.0175e-5
  "example_put_contract"             => Dict{String, Any}("day"=>Dict{String, A…
  "number_of_trading_periods"        => 120
  "put_option_model"                 => MyAmericanPutContractModel(260.0, 1, 0.…
  "β"                                => 2.32

### Constants
Now that we have loaded the save file, we specify values for the constants used in the example by passing the respective keys to the variable `my_simulation_dict::Dict{String, Any}.`
- The variable `DTE::Float64` represents the days to the expiration of the options contracts (measured in days), `Δt::Float64` represents the time step used in the simulations. In this case, we are using 1-minute aggregate data (measured in years). The variable `number_of_trading_periods::Int64` represents the number of trading periods we explore, i.e., the number of minutes that we want to simulate, and `T::Float64` denotes the length of the options contracts (measured in years).
- Next, we retrieve information about `TSLA` and the parameters for the binomial lattice used in the simulation. The variable `β::Float64` represents the beta value for `TSLA`, `h::Int64` represents the number of levels of the pricing tree, the variable `risk_free_rate::Float64` represents the risk-free rate (in this case assumed to be the yield on a 10-year US Treasury Note), the variable `number_of_samples::Int64` specifies the number of samples (alternative futures) we will generate in the simulation, and finally, `sample_synthetic_index::Int64` is a randomly chosen index which specifies which sample we'll use in the simulation.

In [9]:
DTE = my_simulation_dict["DTE"]; # data from mid-day, add a little something something so we recover the midpoint premium (ish) (all parameters fixed)
Δt = my_simulation_dict["Δt"]; # 1-min trading periods
number_of_trading_periods = my_simulation_dict["number_of_trading_periods"]; # How many trading periods will we simulate?
T = my_simulation_dict["T"]; # Duration of trading periods in option years
β = my_simulation_dict["β"]; # TSLA 5y monthly beta value 06-25-24
h = my_simulation_dict["h"]; # 6-points per trading period
risk_free_rate = my_simulation_dict["risk_free_rate"]; # yield 10-year treasury note on 13-Jun-2024
number_of_samples = my_simulation_dict["number_of_samples"]; # number of samples to generate
sample_synthetic_index_true = my_simulation_dict["sample_synthetic_index"]; # generate random synthetic index to explore (must be <= number_of_samples)

568

Finally, let's randomly generate an index associated with the model, i.e., the column of data that the model thinks is the true price and volatility time series. We save this index in the `sample_synthetic_index_model::Int64` variable:

In [34]:
sample_synthetic_index_model = nothing;
should_we_keep_looping = true;
while (should_we_keep_looping == true)
    sample_synthetic_index_model = rand(1:number_of_samples);
    if (sample_synthetic_index_true != sample_synthetic_index_model)
        should_we_keep_looping = false;
    end
end

### Contracts
We can load the previous contract data by passing in the `example_put_contract` key to the `my_simulation_dict::Dict{String, Any}` variable. We save the contract in the `example_put_contract::Dict{String, Any}` variable:

In [12]:
example_put_contract = my_simulation_dict["example_put_contract"]

Dict{String, Any} with 9 entries:
  "day"                => Dict{String, Any}("change"=>0, "high"=>25.03, "change…
  "details"            => Dict{String, Any}("contract_type"=>"put", "ticker"=>"…
  "last_quote"         => Dict{String, Any}("midpoint"=>23.7, "ask"=>23.8, "tim…
  "last_trade"         => Dict{String, Any}("timeframe"=>"REAL-TIME", "exchange…
  "underlying_asset"   => Dict{String, Any}("timeframe"=>"REAL-TIME", "ticker"=…
  "greeks"             => Dict{String, Any}("theta"=>-0.198501, "gamma"=>0.0063…
  "open_interest"      => 2878
  "implied_volatility" => 0.605621
  "break_even_price"   => 236.3

Finally, we set some data from this contract, in particular, the initial share price in the `Sₒ::Float64` variable and the implied volatility `IVₒ::Float64` variable:

### Share price and volatility simulation data
Since we saved the previous share price and volatility simulation data, we can load these data arrays from the `my_simulation_dict::Dict{String, Any}` variable (where we do not have to repeat the simulations and analysis underlying the simulations). First, retrieve the `number_of_trading_periods`$\times$`number_of_paths` array holding possible `TSLA` share price values and save this data in the `samples_share_price::Array{Float64,2}` array:

In [15]:
samples_share_price = my_simulation_dict["samples_share_price"];

Then, retrieve the future implied volatility data from the `my_simulation_dict::Dict{String, Any}` by passing the `samples_IV::String` key to the dictionary. We store the `number_of_trading_periods`$\times$`number_of_samples` volatility array to the  `samples_IV::Array{Float64,2}` variable:

In [17]:
samples_IV = my_simulation_dict["samples_IV"];

Finally, generate `true` and `model` price and volatility datasets. The `true` price and volatility datasets represent the actual data. On the other hand, the `model` price and volatility datasets are the best estimates that the model has access to. Given that `sample_synthetic_index_model`$\neq$`sample_synthetic_index_true`, the `true` and `model` datasets will not be the same.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.